In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, SubsetRandomSampler

# Resizing the images to a common size (224x224) and converting them to tensors
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

# Loading the training dataset and splitting it into training and validation sets (80%, 20%)
trainDataset = datasets.ImageFolder("/kaggle/input/inaturalist12k/Data/inaturalist_12K/train/", transform=transform)
trainIndices, valIndices = train_test_split(list(range(len(trainDataset))), test_size=0.2, random_state=42)
trainSampler, valSampler = SubsetRandomSampler(trainIndices), SubsetRandomSampler(valIndices)

# Function to train the ResNet50 model
def trainNetwork():

    # Creating data loaders for training and validation sets
    trainLoader = DataLoader(trainDataset, batch_size=32, sampler=trainSampler)
    valLoader = DataLoader(trainDataset, batch_size=32, sampler=valSampler)
    
    # Loading the pre-trained ResNet50 model and modifying the fully connected layer for 10 classes
    model = models.resnet50(pretrained=True)
    n_ftrs = model.fc.in_features
    # 10 output classes
    model.fc = torch.nn.Linear(n_ftrs, 10)
    # Freezing all parameters except the fully connected layer for fine-tuning
    for param in model.parameters():
        param.requires_grad = False
    for param in model.fc.parameters():
        param.requires_grad = True
    
    # Defining loss criterion, optimizer, and moving the model to GPU if available
    lossCriteria = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Training loop for 10 epochs
    for i in range(10):
        tempLoss = 0.0
        true = 0
        total = 0
        # Iterating over batches in the training dataset
        for inputs, labels in trainLoader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = lossCriteria(outputs, labels)
            loss.backward()
            optimizer.step()
            tempLoss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            true += predicted.eq(labels).sum().item()

        # Computing training loss and accuracy
        trainLoss = tempLoss/len(trainLoader)
        trainAccuracy = 100*(true/total)
        # Validating the model
        valLoss, valAccuracy = validateNetwork(valLoader, model)
        # Printing training and validation metrics for each epoch
        print(f"epoch {i+1}/10: train loss: {trainLoss}, train accuracy: {trainAccuracy}, val loss: {valLoss}, val accuracy: {valAccuracy}")


# Function to validate the model on the validation set
def validateNetwork(valLoader,model):
    # Set the model to evaluation mode
    model.eval()
    valLoss = 0.0
    true = 0
    total = 0
    lossCriteria = nn.CrossEntropyLoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    with torch.no_grad():
        # Iterating over batches in the validation dataset
        for inputs, labels in valLoader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = lossCriteria(outputs, labels)
            valLoss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            true += predicted.eq(labels).sum().item()

    # Computing validation loss and accuracy
    valLoss /= len(valLoader)
    valAccuracy = 100*(true/total)
    return valLoss, valAccuracy

# Calling the training function
trainNetwork()